In [1]:
import kagglehub
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from joblib import dump, load
from sklearn.model_selection import GridSearchCV
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("fmena14/volcanoesvenus")
print("Path to dataset files:", path)

# List files in the dataset directory
files = os.listdir(path)
print("Files in the dataset directory:", files)

# Assign paths to train and test directories
train_dir = os.path.join(path, 'volcanoes_train')
test_dir = os.path.join(path, 'volcanoes_test')

train_files = os.listdir(train_dir)
print("Sample files in training directory:", train_files)

test_files = os.listdir(test_dir)
print("Sample files in test directory:", test_files[:5])


# Load sample data from training images and labels
train_images_path = os.path.join(train_dir, 'train_images.csv')
train_labels_path = os.path.join(train_dir, 'train_labels.csv')

train_images = pd.read_csv(train_images_path, header=None)
train_labels = pd.read_csv(train_labels_path)


train_images_scaled = train_images.values / 255 # normalize pixel values to [0,1]

# SPLIT train into train and validation set (use test only for final evalutation)
X_train, X_val, y_train, y_val = train_test_split(train_images_scaled, 
                                                    train_labels['Volcano?'], 
                                                    test_size=0.2,
                                                   random_state = 2024)

100%|██████████| 231M/231M [00:22<00:00, 10.6MB/s] 

Extracting files...


Path to dataset files: C:\Users\jakob\.cache\kagglehub\datasets\fmena14\volcanoesvenus\versions\1
Files in the dataset directory: ['Volcanoes_test', 'Volcanoes_train']
Sample files in training directory: ['train_images.csv', 'train_labels.csv']
Sample files in test directory: ['test_images.csv', 'test_labels.csv']


In [3]:
from sklearn.ensemble import GradientBoostingClassifier

# Create and train the Gradient Boosting model
model_gb = GradientBoostingClassifier(n_estimators=100,
                                      learning_rate=0.2,
                                      max_depth=3,
                                      random_state=2024)
model_gb.fit(X_train, y_train)

# Make predictions
y_pred_gb = model_gb.predict(X_val)

# Evaluate accuracy
accuracy_gb = accuracy_score(y_val, y_pred_gb)
print(f"Gradient Boosting Accuracy: {accuracy_gb*100:.2f}")


Gradient Boosting Accuracy: 93.57


In [ ]:
import xgboost as xgb
model_xgb = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=.5,
    max_depth=3,
    random_state=2024,
    n_jobs = -1
)

model_xgb.fit(X_train, y_train)

y_pred_xgb = model_xgb.predict(X_val)
accuracy_xgb = accuracy_score(y_val, y_pred_xgb)
print(f'Extreme grtadient boosting accuracy: {accuracy_xgb*100:.2f}')

Extreme grtadient boosting accuracy: 94.36


: 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100],  # Focus on fewer stages
    'learning_rate': [0.05, 0.1, 0.2],  # Adjust sensitivity
    'max_depth': [3, 5, 7],  # Test key depths
    'subsample': [0.8, 1.0],  # Focus on critical subsampling levels
}

# Create the Gradient Boosting model
gb_model = GradientBoostingClassifier(random_state=2024)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid, 
                           scoring='accuracy', cv=5, verbose=3)

# Perform the grid search
grid_search.fit(X_train, y_train)

# Best hyperparameters
print("Best hyperparameters:", grid_search.best_params_)

# Use the best model to make predictions
best_gb_model = grid_search.best_estimator_
y_pred_gb = best_gb_model.predict(X_val)

# Evaluate accuracy
accuracy_gb = accuracy_score(y_val, y_pred_gb)
print(f"Tuned Gradient Boosting Accuracy: {accuracy_gb * 100:.2f}")


In [11]:
import xgboost as xgb

param_space = {
    #'n_estimators' : [50, 100],
    'max_depth' : [3,5,7],
    'learning_rate' : [0.1,0.01, 0.001],
   #'subsample' : [0.5, 0.7, 1],
}

xgb_model = xgb.XGBClassifier(device='cuda', max_depth=3)

grid_search_xgb = GridSearchCV(estimator=xgb_model, param_grid=param_space,
                               scoring='accuracy', cv=5, verbose=3, n_jobs=-1)

grid_search_xgb.fit(X_train, y_train)

print("Best hyperparameters:", grid_search_xgb.best_params_)

# Use the best model to make predictions
best_xgb_model = grid_search_xgb.best_estimator_
y_pred_xgb = best_xgb_model.predict(X_val)

# Evaluate accuracy
from utils import  format_param_grid
accuracy_xgb = accuracy_score(y_val, y_pred_xgb)
print(f"Tuned Gradient Boosting Accuracy: {accuracy_xgb * 100:.2f}")
dump(best_xgb_model, f'models/best_xgb_model_val_acc_{format_param_grid(accuracy_xgb)* 100:.2f}.joblib')


Fitting 5 folds for each of 9 candidates, totalling 45 fits


KeyboardInterrupt: 